In [1]:
import GCRCatalogs
import matplotlib.pyplot as plt
import pandas as pd
import glob
import numpy as np
from astropy import table
import seaborn as sns

In [2]:
cosmo = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_image")

In [18]:
def collect_snana_outputs(path): #path may be single or list
    if type(path)==str:
        headfiles = sorted(glob.glob(path+ '*HEAD.FITS*'))
        photfiles = sorted(glob.glob(path+ '*PHOT.FITS*'))
    elif type(path)==list:
        headfiles = []
        photfiles = []
        for p in path:
            headfiles.extend(sorted(glob.glob(p+ '*HEAD.FITS*')))
            photfiles.extend(sorted(glob.glob(p+ '*HEAD.FITS*')))
    c=0
    for file in headfiles[:2]: # don't read more than 10
        if c==0:
            headfile = table.Table.read(file, format='fits').to_pandas()
        else:
            headfile=headfile.append(table.Table.read(file, format='fits').to_pandas())
        c+=1
    return headfile
        
def pull_cosmodc2_props(quantities, indices):
    filters=[(lambda x: np.isin(x, indices), 'galaxy_id')]
    quants = quantities+['galaxy_id']
    gals = cosmo.get_quantities(quants, filters)
    return gals

In [6]:
ls /global/cscratch1/sd/kessler/SNANA_LSST_SIM/MLAG_GP_ELASTICC_TEST_LSST_WFD/

MLAG_GP_ELASTICC_TEST_LSST_WFD_AGN/
MLAG_GP_ELASTICC_TEST_LSST_WFD_KN_B19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_KN_K17/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SLSN-I/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNII+HostXT_V19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNII-NMF/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNII-Templates/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIIb+HostXT_V19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIIn+HostXT_V19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIIn-MOSFIT/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIa-91bg/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIa-SALT2/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIax/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIb+HostXT_V19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIb-Templates/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIc+HostXT_V19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIc-Templates/
MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIcBL+HostXT_V19/
MLAG_GP_ELASTICC_TEST_LSST_WFD_TDE/


In [43]:
path = "/global/cscratch1/sd/kessler/SNANA_LSST_SIM/MLAG_GP_ELASTICC_TEST_LSST_WFD/"
snia = collect_snana_outputs(path+"MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIa-SALT2/")


In [44]:
sniax = collect_snana_outputs(path+"MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIax/")


In [45]:
sn91bg = collect_snana_outputs(path+"MLAG_GP_ELASTICC_TEST_LSST_WFD_SNIa-91bg/")

In [46]:
snia   = snia.sample(n=10000)
sniax  = sniax.sample(n=10000)
sn91bg = sn91bg.sample(n=10000)

In [47]:
quantities= ['Mag_true_u_sdss_z0','Mag_true_r_sdss_z0']
snia = pull_cosmodc2_props(quantities, snia['HOSTGAL_OBJID'])


In [48]:
sniax = pull_cosmodc2_props(quantities, sniax['HOSTGAL_OBJID'])

In [49]:
sn91bg =  pull_cosmodc2_props(quantities, sn91bg['HOSTGAL_OBJID'])

In [50]:
np.average(snia['Mag_true_u_sdss_z0']-snia['Mag_true_r_sdss_z0']), np.std(snia['Mag_true_u_sdss_z0']-snia['Mag_true_r_sdss_z0'])/np.sqrt(len(snia['Mag_true_r_sdss_z0']))


(1.5546043, 0.004474265293252125)

In [51]:
np.average(sniax['Mag_true_u_sdss_z0']-sniax['Mag_true_r_sdss_z0']), np.std(sniax['Mag_true_u_sdss_z0']-sniax['Mag_true_r_sdss_z0'])/np.sqrt(len(sniax['Mag_true_u_sdss_z0']))

(1.1050878, 0.004253949642383669)

In [52]:
np.average(sn91bg['Mag_true_u_sdss_z0']-sn91bg['Mag_true_r_sdss_z0']), np.std(sn91bg['Mag_true_u_sdss_z0']-sn91bg['Mag_true_r_sdss_z0'])/np.sqrt(len(sn91bg['Mag_true_u_sdss_z0']))


(1.8446063, 0.003967806262442249)

In [56]:
# hakobyan 91bg vs normal
print(2.23-1.86)
# ours
print(1.8446063-1.5546043)

0.3699999999999999
0.29000199999999987


In [57]:
# hakobyan normal vs Iax
print(1.86-1.47)
# ours
print(1.5546043-1.1050878)

0.3900000000000001
0.4495165000000001


In [59]:
(1.86-1.55)/np.sqrt(.03**2+.004**2)

10.242688307053895

In [60]:
(1.47-1.11)/np.sqrt(.1**2+.004**2)

3.5971234513984403

In [61]:
(2.23-1.84)/np.sqrt(.05**2+.004**2)

7.775159172581772